# SMS Spam Collection Dataset

### 1. DESCRIPTION
--------------

The SMS Spam Collection v.1 (hereafter the corpus) is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam. 

#### 1.1. Compilation
----------------

This corpus has been collected from free or free for research sources at the Web:

- A collection of between 425 SMS spam messages extracted manually from the Grumbletext Web site. This is a UK forum in which cell phone users make public claims about SMS spam messages, most of them without reporting the very spam message received. The identification of the text of spam messages in the claims is a very hard and time-consuming task, and it involved carefully scanning hundreds of web pages. The Grumbletext Web site is: http://www.grumbletext.co.uk/
- A list of 450 SMS ham messages collected from Caroline Tag's PhD Theses available at http://etheses.bham.ac.uk/253/1/Tagg09PhD.pdf
- A subset of 3,375 SMS ham messages of the NUS SMS Corpus (NSC), which is a corpus of about 10,000 legitimate messages collected for research at the Department of Computer Science at the National University of Singapore. The messages largely originate from Singaporeans and mostly from students attending the University. These messages were collected from volunteers who were made aware that their contributions were going to be made publicly available. The NUS SMS Corpus is avalaible at: http://www.comp.nus.edu.sg/~rpnlpir/downloads/corpora/smsCorpus/
- The amount of 1,002 SMS ham messages and 322 spam messages extracted from the SMS Spam Corpus v.0.1 Big created by Jos?Mar? G?ez Hidalgo and public available at: http://www.esp.uem.es/jmgomez/smsspamcorpus/


#### 1.2. Statistics
---------------

There is one collection:

- The SMS Spam Collection v.1 (text file: smsspamcollection) has a total of 4,827 SMS legitimate messages (86.6%) and a total of 747 (13.4%) spam messages.


#### 1.3. Format
-----------

The files contain one message per line. Each line is composed by two columns: one with label (ham or spam) and other with the raw text. Here are some examples:

ham   What you doing?how are you?
ham   Ok lar... Joking wif u oni...
ham   dun say so early hor... U c already then say...
ham   MY NO. IN LUTON 0125698789 RING ME IF UR AROUND! H*
ham   Siva is in hostel aha:-.
ham   Cos i was out shopping wif darren jus now n i called him 2 ask wat present he wan lor. Then he started guessing who i was wif n he finally guessed darren lor.
spam   FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! ubscribe6GBP/ mnth inc 3hrs 16 stop?txtStop
spam   Sunshine Quiz! Win a super Sony DVD recorder if you canname the capital of Australia? Text MQUIZ to 82277. B
spam   URGENT! Your Mobile No 07808726822 was awarded a L2,000 Bonus Caller Prize on 02/09/03! This is our 2nd attempt to contact YOU! Call 0871-872-9758 BOX95QU

Note: messages are not chronologically sorted.


### 2. USAGE
--------

We offer a comprehensive study of this corpus in the following paper that is under review. This work presents a number of statistics, studies and baseline results for several machine learning methods.

[1] Almeida, T.A., G?ez Hidalgo, J.M., Yamakami, A. Contributions to the study of SMS Spam Filtering: New Collection and Results. Proceedings of the 2011 ACM Symposium on Document Engineering (ACM DOCENG'11), Mountain View, CA, USA, 2011. (Under review)


### 3. ABOUT
--------

The corpus has been collected by Tiago Agostinho de Almeida (http://www.dt.fee.unicamp.br/~tiago) and Jos?Mar? G?ez Hidalgo (http://www.esp.uem.es/jmgomez).

We would like to thank Dr. Min-Yen Kan (http://www.comp.nus.edu.sg/~kanmy/) and his team for making the NUS SMS Corpus available. See: http://www.comp.nus.edu.sg/~rpnlpir/downloads/corpora/smsCorpus/. He is currently collecting a bigger SMS corpus at: http://wing.comp.nus.edu.sg:8080/SMSCorpus/

### 4. LICENSE/DISCLAIMER
---------------------

We would appreciate if:

- In case you find this corpus useful, please make a reference to previous paper and the web page: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/ in your papers, research, etc.
- Send us a message to tiago@dt.fee.unicamp.br in case you make use of the corpus.

The SMS Spam Collection v.1 is provided for free and with no limitations excepting:

1. Tiago Agostinho de Almeida and Jos?Mar? G?ez Hidalgo hold the copyrigth (c) for the SMS Spam Collection v.1.

2. No Warranty/Use At Your Risk. 

3. Limitation of Liability.


## Import modules

In [1]:
from pyspark.sql import SparkSession


### Tokenzier & 토컨 개수 계산

In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
# col: Returns a Column based on the given column name
# udf: user defined function

In [3]:
countTokenizer = udf(lambda w: len(w), IntegerType())

``` Python
@udf(IntegerType())
countTokenizer(w):
    return len(w)
```

### Stop words removal

In [21]:
from pyspark.ml.feature import StopWordsRemover

### Term Freq-Inverse doc freq TF-IDF


In [6]:
from pyspark.ml.feature import IDF

## 데이터 읽기 및 변수명(columns) 변경

In [8]:
spark = SparkSession.builder.appName("nlp_nb").getOrCreate()

In [44]:
df = spark.read.csv("SMSSpamCollection",inferSchema=True,sep="\t")
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [45]:
df = df.withColumnRenamed("_c0","label").withColumnRenamed("_c1","messages")
df.show()

+-----+--------------------+
|label|            messages|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [46]:
from pyspark.sql.functions import length

In [47]:
df = df.withColumn("length",length(df["messages"]))
df.show()

+-----+--------------------+------+
|label|            messages|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [48]:
df.groupby("label").mean().show()

+-----+-----------------+
|label|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [49]:
### Feature transformation
from pyspark.ml.feature import StringIndexer

In [50]:
tokenizer = Tokenizer(inputCol="messages",outputCol="tokened")
stop_word_remover = StopWordsRemover(inputCol="tokened",outputCol="stoped")
count_vec = CountVectorizer(inputCol="stoped",outputCol="c_vec")
idf = IDF(inputCol="c_vec",outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol="label",outputCol="label_01")

In [51]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

cleaned = VectorAssembler(inputCols=['tf_idf','length'],outputCol="features")

In [57]:
### Pipeline
from pyspark.ml import Pipeline
파이프라인 = Pipeline(stages=[ham_spam_to_num, tokenizer,stop_word_remover,count_vec,idf,cleaned])

In [58]:
전처리기 = 파이프라인.fit(df)
전처리_df = 전처리기.transform(df)
전처리_df.show()

+-----+--------------------+------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|            messages|length|label_01|             tokened|              stoped|               c_vec|              tf_idf|            features|
+-----+--------------------+------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|     0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|     0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|   155|     1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|     0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,8

In [ ]:
### Train model and evaluation
최종_df = 전처리_df.select(['label_01','features']).withColumnRenamed("label_01","label")

In [59]:
(train_df, test_df) = 최종_df.randomSplit([.75,.25], seed= 316)

In [60]:
train_df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[0,1,2,13,...|
|  0.0|(13424,[0,1,2,41,...|
|  0.0|(13424,[0,1,3,9,1...|
|  0.0|(13424,[0,1,4,50,...|
|  0.0|(13424,[0,1,5,20,...|
|  0.0|(13424,[0,1,7,8,1...|
|  0.0|(13424,[0,1,7,8,1...|
|  0.0|(13424,[0,1,7,15,...|
|  0.0|(13424,[0,1,9,14,...|
|  0.0|(13424,[0,1,9,14,...|
|  0.0|(13424,[0,1,11,32...|
|  0.0|(13424,[0,1,12,33...|
|  0.0|(13424,[0,1,14,18...|
|  0.0|(13424,[0,1,14,31...|
|  0.0|(13424,[0,1,14,78...|
|  0.0|(13424,[0,1,15,20...|
|  0.0|(13424,[0,1,17,19...|
|  0.0|(13424,[0,1,18,20...|
|  0.0|(13424,[0,1,30,12...|
|  0.0|(13424,[0,1,43,69...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
### Naive Bayes Model
from pyspark.ml.classification import NaiveBayes, LogisticRegression, RandomForestClassifier, GBTClassifier, LinearSVC
nb = NaiveBayes()
## labelCol:str="label", featuresCol:str = "features", predictionCol:str="prediction"

In [62]:
적합모형 = nb.fit(train_df)
적합결과 = 적합모형.transform(test_df)

In [63]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

eval = MulticlassClassificationEvaluator()
acc = eval.evaluate(적합결과)
print(f"Accuracy:{acc*100}")



Accuracy:91.94272940617914
